# 부록 10.2.1: 첫 번째 간단한 도구

이전 강의에서 우리는 도구 사용 워크플로우를 살펴보았습니다. 이제 실제로 도구 사용의 간단한 예를 구현할 시간입니다. 복습하자면, 도구 사용 프로세스에는 최대 4단계가 있습니다.

1. **Claude에게 도구와 사용자 프롬프트 제공:** (API 요청)
    * Claude가 액세스할 수 있는 도구 세트를 정의하고, 이름, 설명 및 입력 스키마를 포함합니다.
    * 이러한 도구 중 하나 이상을 사용해야 할 수 있는 사용자 프롬프트를 제공합니다.

2. **Claude가 도구 사용:** (API 응답)
    * Claude는 사용자 프롬프트를 평가하고 사용 가능한 도구 중 하나라도 사용자의 쿼리나 작업에 도움이 되는지 판단합니다. 그렇다면 어떤 도구를 사용할지와 어떤 입력을 사용할지도 결정합니다.
    * Claude는 적절한 형식의 도구 사용 요청을 출력합니다.
    * API 응답에는 `stop_reason`이 `tool_use`로 표시되어 Claude가 외부 도구를 사용하려 한다는 것을 나타냅니다.

3. **도구 입력 추출, 코드 실행 및 결과 반환:** (API 요청)
    * 클라이언트 측에서 Claude의 도구 사용 요청에서 도구 이름과 입력을 추출해야 합니다.
    * 클라이언트 측에서 실제 도구 코드를 실행합니다.
    * `tool_result` 콘텐츠 블록이 포함된 새로운 사용자 메시지로 대화를 계속하여 Claude에게 결과를 반환합니다.

4. **Claude가 도구 결과를 사용하여 응답 작성:** (API 응답)
    * 도구 결과를 받은 후 Claude는 그 정보를 사용하여 원래 사용자 프롬프트에 대한 최종 응답을 작성합니다.

우리는 Claude와 "대화"하는 것만으로 시작할 것입니다(곧 더 흥미로운 예제가 올 것이니 걱정하지 마세요!). 즉, 아직 4단계는 신경 쓰지 않겠습니다. Claude에게 질문에 답하도록 요청하고, Claude는 그 질문에 답하기 위해 도구를 사용하겠다고 요청할 것입니다. 그리고 우리는 도구 입력을 추출하고, 코드를 실행하고, 결과 값을 반환할 것입니다.

오늘날의 대형 언어 모델은 다음 코드에서 볼 수 있듯이 수학 연산에 어려움을 겪고 있습니다.

Claude에게 "1984135를 9343116으로 곱하라"고 요청합니다.

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError
session = boto3.Session() # create a boto3 session to dynamically get and set the region name
region = session.region_name

# Import the hints module from the utils package
from utils import hints

#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

In [ ]:
converse_api_params = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",  # Specify the model ID to use
    "messages": [{"role": "user", "content": [{"text": "Multiply 1984135 by 9343116. Only respond with the result"}]}],
    "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
}

response = bedrock_client.converse(**converse_api_params)

print(response['output']['message']['content'][0]['text'])

위 코드를 여러 번 실행하면 다른 답변이 나올 수 있지만, Claude가 응답한 한 가지 답변은 다음과 같습니다.

```
18593367726060
```

실제 정확한 답은 다음과 같습니다.

```
18538003464660
```
Claude는 `55364261400`만큼 약간 벗어났습니다!

## 도구 사용의 구원!

Claude는 복잡한 수학 계산을 잘하지 못하므로, 계산기 도구에 액세스할 수 있도록 Claude의 기능을 향상시켜 보겠습니다.

프로세스를 설명하는 간단한 다이어그램은 다음과 같습니다:

![chickens_calculator.png](./images/chickens_calculator.png)
첫 번째 단계는 실제 계산기 함수를 정의하고 Claude와 독립적으로 작동하는지 확인하는 것입니다. 다음과 같은 세 가지 인수를 예상하는 매우 간단한 함수를 작성할 것입니다:
* "add" 또는 "multiply"와 같은 연산
* 두 개의 피연산자

기본 구현은 다음과 같습니다:

In [ ]:
def calculator(operation, operand1, operand2):
    if operation == "add":
        return operand1 + operand2
    elif operation == "subtract":
        return operand1 - operand2
    elif operation == "multiply":
        return operand1 * operand2
    elif operation == "divide":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")

이 간단한 함수의 유틸리티는 `234 + 213` 또는 `3 * 9`와 같은 간단한 식만 처리할 수 있기 때문에 상당히 제한적입니다. 여기서 중요한 점은 매우 간단한 교육 예제를 통해 도구 작업 프로세스를 살펴보는 것입니다.

함수를 테스트하고 제대로 작동하는지 확인해 보겠습니다.

In [ ]:
calculator("add", 10, 3)

In [ ]:
calculator("divide", 200, 25)

다음 단계는 우리의 도구를 정의하고 Claude에게 알려주는 것입니다. 도구를 정의할 때는 매우 구체적인 형식을 따릅니다. 각 도구 정의에는 다음이 포함됩니다:

* `name`: 도구의 이름입니다. 정규식 ^[a-zA-Z0-9_-]{1,64}$와 일치해야 합니다.
* `description`: 도구가 무엇을 하는지, 언제 사용해야 하는지, 어떻게 동작하는지에 대한 자세한 평문 설명입니다.
* `input_schema`: 도구에 대한 예상 매개변수를 정의하는 JSON 스키마 객체입니다.

JSON 스키마에 익숙하지 않으신가요? [여기서 자세히 알아보세요](https://json-schema.org/learn/getting-started-step-by-step).

가상의 도구에 대한 간단한 예시입니다:

```json
{
  "tools": [
    {
      "toolSpec": {
        "name": "send_email",
        "description": "지정된 수신자에게 주어진 제목과 본문으로 이메일을 보냅니다.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "to": {
                "type": "string",
                "description": "수신자의 이메일 주소"},
              "subject": {
                "type": "string",
                "description": "이메일의 제목 줄"},
              "body": {
                "type": "string",
                "description": "이메일 메시지의 내용"}
            },
            "required": ["to", "subject", "body"]
          }
        }
      }
    }
  ]
}
```

이 `send_email` 도구는 다음 입력을 예상합니다:
* `to`는 필수 문자열입니다.
* `subject`는 필수 문자열입니다.
* `body`는 필수 문자열입니다.


`search_product`라는 도구에 대한 또 다른 도구 정의입니다:

```json
{
  "tools": [
    {
      "toolSpec": {
        "name": "search_product",
        "description": "이름 또는 키워드로 제품을 검색하고 현재 가격과 재고 여부를 반환합니다.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "query": {
                "type": "string",
                "description": "제품 이름 또는 검색 키워드, 예: 'iPhone 13 Pro' 또는 'wireless headphones'"},
              "category": {
                "type": "string",
                "enum": ["electronics", "clothing", "home", "toys", "sports"],
                "description": "검색 결과를 좁히기 위한 제품 카테고리"},
              "max_price": {
                "type": "number",
                "description": "제품의 최대 가격, 검색 결과를 필터링하는 데 사용됩니다"}
            },
            "required": ["query"]
          }
        }
      }
    }
  ]
}
```
이 도구에는 3개의 입력이 있습니다:
* 필수 `query` 문자열로, 제품 이름 또는 검색 키워드를 나타냅니다.
* 선택적 `category` 문자열로, 미리 정의된 값 중 하나여야 하며 검색 결과를 좁히는 데 사용됩니다. 정의에서 `"enum"`을 확인하세요.
* 선택적 `max_price` 숫자로, 특정 가격 이하의 결과를 필터링합니다.

### 우리의 계산기 도구 정의
앞서 작성한 계산기 함수에 대한 해당 도구를 정의해 봅시다. 계산기 함수에는 3개의 필수 인수가 있습니다.
* `operation` - "add", "subtract", "multiply" 또는 "divide"만 가능합니다.
* `operand1` - 숫자여야 합니다.
* `operand2` - 숫자여야 합니다.

도구 정의는 다음과 같습니다:

In [ ]:
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "calculator",
        "description": "A simple calculator that performs basic arithmetic operations.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The arithmetic operation to perform."
              },
              "operand1": {
                "type": "number",
                "description": "The first operand."},
              "operand2": {
                "type": "number",
                "description": "The second operand."}
            },
            "required": ["operation", "operand1", "operand2"]
          }
        }
      }
    }
  ]
}

***

## 연습

다음 함수를 예시로 사용하여 제대로 형식이 지정된 도구 정의를 작성해 봅시다.

In [ ]:
def inventory_lookup(product_name, max_results):
    return "this function doesn't do anything"
    #You do not need to touch this or do anything with it!

이 가상의 `inventory_lookup` 함수는 다음과 같이 호출되어야 합니다:

In [ ]:
inventory_lookup("AA batteries", 4)

inventory_lookup("birthday candle", 10)

당신의 과제는 해당하는 제대로 형식이 지정된 도구 정의를 작성하는 것입니다. 도구 정의에서 두 인수 모두 필수로 가정하세요.

### 시작 도구 정의 템플릿 ###

```json
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "inventory_lookup",
        "description": " ",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              " ": {
                "type": "string",
                "description": " "
              },
              " ": {
                "type": "number",
                "description": " "
              },
            },
            "required": [" ", " "]
          }
        }
      }
    }
  ]
}
```

***

### Claude에게 우리의 도구 제공하기
이제 앞서 다룬 계산기 함수로 돌아갑시다. 이 시점에서 Claude는 계산기 도구에 대해 전혀 모릅니다! 그것은 단지 작은 Python 딕셔너리일 뿐입니다. Claude에 요청을 할 때 우리는 도구 목록을 전달하여 Claude에게 "알려줄" 수 있습니다. 지금 해봅시다:

In [ ]:
converse_api_params = {
    "modelId": modelId,
    "messages": [{"role": "user", "content": [{"text": "Multiply 1984135 by 9343116. Only respond with the result."}]}],
    "toolConfig": toolConfig, # provide Claude with details about our calculator tool
}

response = bedrock_client.converse(**converse_api_params)

다음으로 Claude가 우리에게 돌려준 응답을 살펴봅시다:

In [ ]:
response['output']

```
{'message': {'role': 'assistant',
  'content': [{'toolUse': {'toolUseId': 'tooluse_YOMRWNbNQuCP-BR16tY6mw',
     'name': 'calculator', <---------------------------------------------- Claude는 계산기 도구를 사용하길 원합니다.
     'input': {'operand1': 1984135,
      'operand2': 9343116,
      'operation': 'multiply'}}}]}}
```

우리의 응답이 평소와 다르게 보인다는 것을 알 수 있습니다! 구체적으로, 일반적인 `Message`가 아닌 `ToolsMessage`를 받고 있습니다.

또한 `response['stopReason']`을 확인하면 Claude가 도구를 사용할 때가 되었다고 판단하여 중지했음을 알 수 있습니다.


In [ ]:
response['stopReason']

`response['output']['message']['content']`에는 `ToolUseBlock`이 포함된 목록이 포함되어 있으며, 여기에는 도구 이름과 입력에 대한 정보가 포함되어 있습니다:

In [ ]:
response['output']['message']['content'][-1]

In [ ]:
tool_use = response['output']['message']['content'][-1]
tool_name = tool_use['toolUse']['name']
tool_inputs = tool_use['toolUse']['input']

print("The Tool Name Claude Wants To Call:", tool_name)
print("The Inputs Claude Wants To Call It With:", tool_inputs)

다음 단계는 Claude가 제공한 도구 이름과 입력을 사용하여 앞서 작성한 계산기 함수를 실제로 호출하는 것입니다. 그러면 최종 답변을 얻을 수 있습니다!

In [ ]:
operation = tool_inputs["operation"]
operand1 = tool_inputs["operand1"]
operand2 = tool_inputs["operand2"]

result = calculator(operation, operand1, operand2)
print("RESULT IS", result)

우리는 `18538003464660`의 정확한 답을 얻었습니다!!! Claude에 의존하여 수학 계산을 하는 대신, 우리는 단순히 Claude에게 질문하고 필요한 경우 사용할 수 있는 도구에 접근할 수 있게 합니다.  

#### 중요 참고사항
Claude에게 도구 사용이 필요하지 않은 것을 요청하는 경우, 이 경우 수학이나 계산과 관련이 없는 경우, 우리는 Claude가 정상적으로 응답하기를 원할 것입니다. Claude는 대개 그렇게 할 것이지만, 때때로 Claude는 도구를 사용하는 것에 매우 열정적입니다! 

다음은 Claude가 계산기를 사용하는 것이 적절하지 않음에도 불구하고 계산기를 사용하려고 하는 예시입니다. Claude에게 "에메랄드의 색깔은 무엇입니까?"라고 물어보면 어떻게 될지 봅시다.

In [ ]:
converse_api_params = {
    "modelId": modelId,
    "messages": [{"role": "user", "content": [{"text":"What color are emeralds?"}]}],
    "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    "toolConfig": toolConfig,
}

response = bedrock_client.converse(**converse_api_params)

In [ ]:
response['output']['message']['content']

Claude는 다음과 같이 응답합니다: 

```
[{'toolUse': {'toolUseId': 'tooluse_PM0i2kehQnOq9gcRFa8QEg',
   'name': 'calculator',
   'input': {'operand1': 0, 'operand2': 0, 'operation': 'add'}}}]

```
Claude는 우리에게 계산기 도구를 호출하라고 합니다? 매우 쉬운 해결책은 프롬프트를 조정하거나 다음과 같은 시스템 프롬프트를 추가하는 것입니다: `도구에 접근할 수 있지만, 필요한 경우에만 사용하세요. 도구가 필요하지 않다면 정상적으로 응답하세요`.

In [ ]:
converse_api_params = {
    "modelId": modelId,
    "system": [{"text": "You have access to tools, but only use them when necessary. If a tool is not required, respond as normal"}],
    "messages": [{"role": "user", "content": [{"text":"What color are emeralds?"}]}],
    "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    "toolConfig": toolConfig,
}

response = bedrock_client.converse(**converse_api_params)

In [ ]:
response['output']['message']['content'][0]['text']

이제 Claude는 적절한 내용으로 응답하고 도구 사용이 적절하지 않을 때 억지로 도구를 사용하려 하지 않습니다. 우리가 받은 새로운 응답은 다음과 같습니다: 

```
'에메랄드는 녹색입니다.'
```

또한 `stopReason`이 이제 `tool_use`가 아닌 `end_turn`임을 알 수 있습니다.

In [ ]:
response['stopReason']

***

### 전체 정리

In [ ]:
import re
import boto3
import json
from botocore.exceptions import ClientError


def calculator(operation, operand1, operand2):
    if operation == "add":
        return operand1 + operand2
    elif operation == "subtract":
        return operand1 - operand2
    elif operation == "multiply":
        return operand1 * operand2
    elif operation == "divide":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")


toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "calculator",
        "description": "A simple calculator that performs basic arithmetic operations.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "operation": {
                "type": "string",
                "enum": [
                  "add", "subtract", "multiply", "divide"],
                "description": "The arithmetic operation to perform."
              },
              "operand1": {
                "type": "number",
                "description": "The first operand."
              },
              "operand2": {
                "type": "number",
                "description": "The second operand."
              }
            },
            "required": [
              "operation", "operand1", "operand2"]
          }
        }
      }
    }
  ]
}


bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)


def prompt_claude(prompt):

    messages = [{"role": "user", "content": [{"text": prompt}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": "You have access to tools, but only use them when necessary. If a tool is not required, respond as normal"}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
        "toolConfig": toolConfig,
    }

    response = bedrock_client.converse(**converse_api_params)

    if response['stopReason'] == "tool_use":
        tool_use = response['output']['message']['content'][-1]
        tool_name = tool_use['toolUse']['name']
        tool_inputs = tool_use['toolUse']['input']

        if tool_name == "calculator":
            print("Claude wants to use the calculator tool")
            operation = tool_inputs["operation"]
            operand1 = tool_inputs["operand1"]
            operand2 = tool_inputs["operand2"]

            try:
                result = calculator(operation, operand1, operand2)
                print("Calculation result is:", result)
            except ValueError as e:
                print(f"Error: {str(e)}")

    elif response['stopReason'] == "end_turn":
        print("Claude didn't want to use a tool")
        print("Claude responded with:")
        print(response['output']['message']['content'][0]['text'])


In [ ]:
prompt_claude("I had 23 chickens but 2 flew away.  How many are left?")

In [ ]:
prompt_claude("What is 201 times 2")

In [ ]:
prompt_claude("Write me a haiku about the ocean")

*** 

## 연습

Claude를 사용하여 연구 보조 도구를 구축하는 것이 귀하의 과제입니다. 사용자는 연구하고 싶은 주제를 입력하면 나중에 읽기 위해 마크다운 파일에 Wikipedia 기사 링크 목록이 저장됩니다. Claude에게 직접 기사 URL 목록을 생성하도록 요청할 수 있지만, Claude는 URL 생성에 신뢰할 수 없으며 기사 URL을 허구적으로 만들어낼 수 있습니다. 또한 Claude의 학습 데이터 기준일 이후에 합법적인 기사가 새로운 URL로 이동했을 수 있습니다. 대신 실제 Wikipedia API에 연결되는 도구를 사용하여 이 작업을 수행할 것입니다! 

우리는 Claude에게 Claude가 생성했지만 허구일 수 있는 잠재적 Wikipedia 기사 제목 목록을 받아들이는 도구에 접근할 수 있게 할 것입니다. 우리는 이 도구를 사용하여 Wikipedia에서 실제 Wikipedia 기사 제목과 URL을 찾아 최종 목록이 실제로 존재하는 기사로만 구성되도록 할 수 있습니다. 그런 다음 나중에 읽기 위해 이 기사 URL을 마크다운 파일에 저장할 것입니다.

도움을 주기 위해 두 가지 함수를 제공했습니다:


In [ ]:
import wikipedia
def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles, topic):
    with open("output/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")

첫 번째 함수 `generate_wikipedia_reading_list`는 "하와이의 역사"나 "전 세계의 해적"과 같은 연구 주제와 Claude가 생성할 잠재적 Wikipedia 기사 제목 목록을 전달받습니다. 이 함수는 `wikipedia` 패키지를 사용하여 해당하는 실제 Wikipedia 페이지를 검색하고, 기사의 제목과 URL을 포함하는 딕셔너리 목록을 작성합니다.그런 다음 Wikipedia 기사 데이터 목록과 전반적인 연구 주제를 `add_to_research_reading_file`에 전달합니다. 이 함수는 단순히 Wikipedia 기사에 대한 markdown 링크를 `output/research_reading.md`라는 파일에 추가합니다. 파일 이름은 현재 하드코딩되어 있으며, 이 함수는 해당 파일이 존재한다고 가정합니다. 이 파일은 이 저장소에 있지만, 다른 곳에서 작업할 경우 직접 만들어야 합니다.아이디어는 Claude에게 실제 Wikipedia 기사인지 아닌지 여부에 관계없이 잠재적 기사 제목 목록을 전달하여 `generate_wikipedia_reading_list`를 "호출"하게 하는 것입니다. Claude는 다음과 같은 입력 기사 제목 목록을 전달할 수 있습니다:```py["Piracy", "Famous Pirate Ships", "Golden Age Of Piracy", "List of Pirates", "Pirates and Parrots", "Piracy in the 21st Century"]````generate_wikipedia_reading_list` 함수는 이러한 기사 제목들을 차례로 검토하여 실제로 존재하는 Wikipedia 기사의 실제 제목과 해당 URL을 수집합니다. 그런 다음 `add_to_research_reading_file`을 호출하여 해당 내용을 나중에 참조할 수 있도록 markdown 파일에 작성합니다.

### 최종 목표여러분의 작업은 연구 주제와 원하는 기사 수를 받아들이는 `get_research_help` 함수를 구현하는 것입니다. 이 함수는 Claude를 사용하여 실제로 가능한 Wikipedia 기사 목록을 생성하고 위의 `generate_wikipedia_reading_list` 함수를 호출해야 합니다. 다음은 몇 가지 함수 호출 예시입니다:```pyget_research_help("Pirates Across The World", 7)get_research_help("History of Hawaii", 3)get_research_help("are animals conscious?", 3)```이 3개의 함수 호출 후, 출력 `research_reading.md` 파일은 다음과 같습니다(output/research_reading.md에서 직접 확인해 보세요):![research_reading.png](./images/research_reading.png)

이를 달성하기 위해서는 다음과 같은 작업을 수행해야 합니다:

* `generate_wikipedia_reading_list` 함수에 대한 tool definition을 작성하세요
* `get_research_help` 함수를 구현하세요
    * 특정 주제에 대한 연구 자료를 수집하는 데 도움이 필요하다고 Claude에게 알리고, 생성할 기사 제목 수를 지정하는 prompt를 작성하세요
    * Claude가 액세스할 수 있는 tool에 대해 알려주세요
    * 요청을 Claude에게 보내세요
    * Claude가 tool을 호출했는지 확인하세요. 호출했다면 생성된 기사 제목과 주제를 제공받은 `generate_wikipedia_reading_list` 함수에 전달해야 합니다. 해당 함수는 실제 Wikipedia 기사 링크를 수집한 다음 `add_to_research_reading_file`을 호출하여 링크를 `output/research_reading.md`에 작성합니다
    * `output/research_reading.md`를 열어 작동 여부를 확인하세요!


#### 시작 코드

In [ ]:
# Here's your starter code!
import wikipedia
def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles, topic):
    with open("output/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")

In [ ]:
# ToolSpec starter code
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "generate_wikipedia_reading_list",
        "description": " ",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "": {
                "type": "string",
                "description": ""
              },
              " ": {
                "type": "string",
                "description": " "
              }
            },
            "required": [" ", " "]
          }
        }
      }
    }
  ]
}

❓ toolSpec과 관련된 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_10_2_1_toolspec)

In [ ]:
def get_research_help(research_topic, num_articles=3):
# create your function here
    pass

❓ get_research_help 함수와 관련된 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_10_2_1_code)

In [ ]:
# Test your function
get_research_help("Claude Shannon", 3)